In [3]:
%pylab inline
import numpy as np
import pymc3 as pm
import theano.tensor as tt

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


```stan
data {
int <lower=0> N;
int <lower=0> D;
matrix [N,D] x ;
vector [N] y;
}
parameters {
vector [D] b;
real <lower=0> sigma;
}
model {
y ~ normal(x * b, sigma);
b~normal(0,1);
sigma~gamma(0.5,0.5);
}
generated quantities{
real  log_density;
log_density=normal_lpdf(y |x * b, sigma)+normal_lpdf(b|0,1)+gamma_lpdf(sigma|0.5,0.5)+log(sigma);
}
```

In [4]:
# a linear regression with 10^5 data and 100 variables
N = 10000
D = 100
beta = np.random.randn(D)
x = np.random.randn(N, D)
y = np.dot(x, beta) + np.random.randn(N)*2

In [5]:
with pm.Model() as md:
    b = pm.Normal('b', 0., 1., shape=D)
    sigma = pm.Gamma('sd', .5, .5)
    pm.Normal('y', tt.dot(x, b), sigma, observed=y)

In [6]:
with md:
    approx = pm.ADVI()
    approx.fit(100000)

Average Loss = 21,500:  25%|██▍       | 24914/100000 [01:48<05:28, 228.70it/s]   
Interrupted at 24,922 [24%]: Average Loss = 1.0039e+05
INFO:pymc3.variational.inference:Interrupted at 24,922 [24%]: Average Loss = 1.0039e+05
